In [1]:
!pip install numpy pandas scikit-learn nltk autocorrect contractions


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Authors: The scikit-learn developers
# SPDX-License-Identifier: BSD-3-Clause

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import sklearn
import nltk
nltk.download('perluniprops')
nltk.download('Wordnet')
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize.nist import NISTTokenizer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/anand/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Error loading Wordnet: Package 'Wordnet' not found in
[nltk_data]     index


In [3]:
lines = []
with open("./data/news_category_classification/bbc-text.csv", "r") as f:
    lines = f.readlines()
df_dict = []
for line in lines:
    line_dict = {}
    line_split = line.split(',')
    if line_split[0]!='category':
        line_dict = {'category':line_split[0], 'text':" ".join(line_split[1:])}
        df_dict.append(line_dict)

df = pd.DataFrame.from_dict(df_dict)

In [4]:
df.head(10)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


In [5]:
X_train = df['text']
y_train = df['category'].values
CATEGORIES = ['business', 'entertainment', 'politics', 'sport', 'tech']
y_train = [CATEGORIES.index(x.replace('\n','')) for x in y_train]

In [ ]:
from nltk.metrics.distance  import edit_distance
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
import contractions
from datetime import datetime
from sklearn.svm import LinearSVC
    
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.speller = Speller(fast=True)

    def __call__(self, text):
        start_time = datetime.now()
        contracted_text = contractions.fix(text)
        corrected_text = self.speller(contracted_text)
        corrected_words = RegexpTokenizer(r'\w+').tokenize(text=corrected_text)
        lemma_words = [self.wnl.lemmatize(t) for t in corrected_words]
        end_time = datetime.now()
        #print(text)
        #print(lemma_words)
        #print(end_time-start_time)
        return lemma_words

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=LemmaTokenizer(), stop_words='english',lowercase=True, analyzer='word',ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    #('clf', MultinomialNB(alpha=0.01, force_alpha=False)),
    ('linearsvc', LinearSVC(random_state=0, tol=1e-05))
])

In [8]:
pipeline.fit(X_train, y_train)

/Users/anand/.pyenv/versions/3.11.10/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/anand/.pyenv/versions/3.11.10/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english',
                                 tokenizer=<__main__.LemmaTokenizer object at 0x125a00a90>)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [9]:
y_pred = pipeline.predict(X_train)
print("Classification report:\n\n{}".format(classification_report(y_train, y_pred)))

Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       510
           1       1.00      1.00      1.00       386
           2       1.00      1.00      1.00       417
           3       1.00      1.00      1.00       511
           4       1.00      1.00      1.00       401

    accuracy                           1.00      2225
   macro avg       1.00      1.00      1.00      2225
weighted avg       1.00      1.00      1.00      2225



In [10]:
X_test = pd.read_csv('data/news_category_classification/test-examples.csv')
y_test = X_test['category']
y_test = [CATEGORIES.index(x) for x in y_test]
y_pred = pipeline.predict(X_test['text'])
print("Classification report:\n\n{}".format(classification_report(y_test, y_pred)))

Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       1.00      1.00      1.00        88
           2       1.00      1.00      1.00       100
           3       1.00      1.00      1.00       108
           4       1.00      1.00      1.00        86

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [11]:
df = pd.read_csv('data/retail-products-classification/train.csv')
df['title_desc'] = [str(x['title']) + " | " + str(x['description']) for index,x in df.iterrows()]
df.dropna(inplace=True)
X_train = df['title_desc']
y_train = df['categories'].values

In [12]:
df.columns

Index(['ImgId', 'title', 'description', 'categories', 'title_desc'], dtype='object')

In [13]:
CATEGORIES = ['Electronics', 'Sports & Outdoors',
       'Cell Phones & Accessories', 'Automotive', 'Toys & Games',
       'Tools & Home Improvement', 'Health & Personal Care', 'Beauty',
       'Grocery & Gourmet Food', 'Office Products',
       'Arts, Crafts & Sewing', 'Pet Supplies', 'Patio, Lawn & Garden',
       'Clothing, Shoes & Jewelry', 'Baby',
       'Musical Instruments', 'Industrial & Scientific', 'Baby Products',
       'Appliances', 'All Beauty', 'All Electronics']
y_train = [CATEGORIES.index(x) for x in y_train]
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_train)

print("Classification report:\n\n{}".format(classification_report(y_train, y_pred)))

/Users/anand/.pyenv/versions/3.11.10/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2187
           1       1.00      1.00      1.00      2178
           2       1.00      1.00      1.00      2188
           3       1.00      1.00      1.00      1999
           4       1.00      1.00      1.00      2194
           5       1.00      1.00      1.00      2183
           6       1.00      1.00      1.00      2154
           7       1.00      1.00      1.00      2149
           8       1.00      1.00      1.00      2196
           9       1.00      1.00      1.00      2144
          10       1.00      1.00      1.00      2175
          11       1.00      1.00      1.00      2124
          12       1.00      1.00      1.00      2168
          13       1.00      1.00      1.00      2182
          14       1.00      1.00      1.00      2177
          15       1.00      1.00      1.00      2095
          16       1.00      1.00      1.00      2016
   

In [15]:
X_test = pd.read_csv('data/retail-products-classification/test.csv')
X_test.fillna('',inplace=True)
X_test['title_desc'] = [str(x['title']) + " | " + str(x['description']) for index,x in X_test.iterrows()]
y_pred = pipeline.predict(X_test['title_desc'])
X_test['category'] = y_pred

import os
images = os.listdir('data/retail-products-classification/test/test/')
images_ids = [x.replace('.jpg','') for x in images]

X_test_filter = X_test[X_test['ImgId'].isin(images_ids)]

print(X_test_filter.shape)

(6367, 5)


In [16]:
X_test_filter[['ImgId','category']].to_csv('output/retail_product_classification/submission.csv', index=False)